In [ ]:
import os
import random
import numpy as np
from sklearn.metrics import (f1_score,
                            roc_auc_score,
                            roc_curve,
                            average_precision_score,
                            precision_recall_curve,
                            mean_absolute_error,
                            mean_squared_error,
                            root_mean_squared_error,
                            r2_score)
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
labeled_data_dir = "train_graphs" #caminho dos dados para treinar o modelo
tam = 30
final_model_dnn_path = f"modelos/dnn_model_{tam}.keras" #caminho do modelo final
final_model_value_path = f"modelos/dnn_value_model_{tam}.keras" #caminho dos valores finais

In [ ]:
# F1-Score
def calcular_f1(true_values, predictions, average='binary'):
    return f1_score(true_values, predictions, average=average)

# AUC-ROC
def auc_roc(true_values, predictions):
    fpr, tpr, _ = roc_curve(true_values, predictions)
    roc = roc_auc_score(true_values, predictions)
    return fpr, tpr, roc

# AUC-PR
def auc_pr(true_values, predictions):
    precision, recall, _ = precision_recall_curve(true_values, predictions)
    pr = average_precision_score(true_values, predictions)
    return precision, recall, pr

# IoU
def iou_score(true_values, predictions):
    intersection = np.sum(np.logical_and(true_values, predictions))  # Elementos em comum (1s)
    union = np.sum(np.logical_or(true_values, predictions))          # União de elementos
    
    plt.figure(figsize=(6, 2))
    plt.bar(range(len(true_values)), true_values, color='green', alpha=0.6, label='Real')
    plt.bar(range(len(predictions)), predictions, color='red', alpha=0.4, label='Predito')
    plt.bar(range(len(intersection)), intersection, color='blue', alpha=0.7, label='Interseção')

    plt.xlabel('Vértices')
    plt.ylabel('Presença na Clique')
    plt.title('Visualização da IoU')
    plt.legend()
    plt.show()
    
    return intersection / union if union != 0 else 0.0

# MAE
def mae(true_values, predictions):
    return mean_absolute_error(true_values, predictions)

# MSE
def mse(true_values, predictions):
    return mean_squared_error(true_values, predictions)

# RMSE
def rmse(true_values, predictions):
    return root_mean_squared_error(true_values, predictions)

# R2
def r2(true_values, predictions):
    return r2_score(true_values, predictions)

def plot_predictions(y_true, y_pred):
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x=y_true, y=y_pred, alpha=0.7)
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], color='red', linestyle='--')
    plt.xlabel("Valores Reais (y_test)")
    plt.ylabel("Previsões (y_pred)")
    plt.title("Comparação entre Valores Reais e Previstos")
    plt.show()

In [ ]:
# Pegando todos os arquivos do diretório
files = sorted([os.path.basename(ii) for ii in glob.glob(f"{labeled_data_dir}/*.dimacs")])
_, files_teste = train_test_split(files, test_size=0.1, random_state=42)

In [ ]:
def parse_file_pointer(fp):
    lines = [ll.strip() for ll in fp]
    ii = 0
    labels = []
    res = []
    cli = []
    numLinhas = 0
    while ii < len(lines):
        line = lines[ii]
        #contando o numero de vertices do grafo
        if "cliqueatual" not in line:
            ii += 1
            numLinhas += 1
            continue

        #pegando a clique atual
        if ii+1 >= len(lines):
            break
        line = line[3:]
        spritado = line.split()
        clique = [int(elem) for elem in spritado[1:]]
        if(numLinhas < tam):
            dif = tam - numLinhas
            clique.extend([0]*dif)
        cli.append(clique)

        #criando o vetor de movimento
        line = lines[ii+1]
        sp = line.split()
        mv = int(sp[-1])
        label = [0] * tam
        label[mv-1] = 1
        labels.append(label)

        #lendo o grafo
        cells = []
        for tt in range(numLinhas, 0, -1):
            cell_line = lines[ii - tt][3:]
            cells.extend([int(float(cc)) for cc in cell_line.split(", ")])
            if(numLinhas < tam):
                dif = tam - numLinhas
                cells.extend([0]*dif)
        while len(cells) < tam * tam:
            cells.extend([0]*tam)
        res.append(cells)
        ii += (numLinhas+2)
    labels_v = list(range(len(labels),0, -1))
    return (res, cli, labels, labels_v)

In [ ]:
def estruturar_entrada(batch_input, batch_labels):
    # Dividir a entrada em uma lista de 151 tensores de forma (batch_size, tam)
    batch_input_list = [batch_input[:, i, :] for i in range(batch_input.shape[1])]
            
    # Converter para tensores do TensorFlow
    x_batch = [tf.convert_to_tensor(tensor, dtype=tf.float32) for tensor in batch_input_list]
    input_dict = {f'input_{i}': tensor for i, tensor in enumerate(x_batch)}
    y_batch = np.array(batch_labels)
    return input_dict, y_batch

In [ ]:
def combinar_entrada(res, clique, labels, remaining_batch_input=[], remaining_batch_labels=[]):
    combined_input = np.array([np.hsplit(np.concatenate([res[i], clique[i]]), tam + 1) for i in range(len(clique))])
    if len(remaining_batch_input) != 0:
        combined_input = np.concatenate((remaining_batch_input, combined_input), axis=0)
        labels = remaining_batch_labels + labels
    return combined_input, labels

In [ ]:
def parse_dir(files):
    res = []
    cli = []
    labels = []
    labels_v = []
    random.seed(42)
    random.shuffle(files)
    random.seed()
    for ff in files:
        with open(os.path.join(labeled_data_dir,ff), 'r') as fp:
            rr, cc, ll, ll_v = parse_file_pointer(fp)
            res.extend(rr)
            cli.extend(cc)
            labels.extend(ll)
            labels_v.extend(ll_v)
    return res, cli, labels, labels_v

In [ ]:
def ler_arquivos(batch_files, value):
    res, clique, labels, labels_v = parse_dir(batch_files)
    if value:
        labels = labels_v
    return res, clique, labels

In [ ]:
# Função para testar o modelo
def testar_modelo(modelo, files_test, usar_labels_value):
    res, clique, labels = ler_arquivos(files_test, usar_labels_value)
    combined_input, labels = combinar_entrada(res, clique, labels)
    x_test, y_test = estruturar_entrada(combined_input, labels)
    # Prever usando o modelo
    previsoes = modelo.predict(x_test)
    
    return y_test, previsoes

In [ ]:
print("Carregando o modelo branch...")
modelo_branch = load_model(final_model_dnn_path)
print("Testando o modelo branch...")
y_test, previsoes = testar_modelo(modelo_branch, files_teste, False)

# F1
f1 = f1_score(y_test, previsoes)
print(f'F1-score: {f1:.4f}')

# AUC ROC
fpr, tpr, auc_roc = auc_roc(y_test, previsoes)
# Plotando a curva ROC
plt.figure(figsize=(6, 6))
plt.plot(fpr, tpr, label=f'AUC-ROC = {auc_roc:.2f}')
plt.plot([0, 1], [0, 1], 'k--')  # Linha diagonal (azar)
plt.xlabel('Taxa de Falsos Positivos')
plt.ylabel('Taxa de Verdadeiros Positivos')
plt.title('Curva ROC')
plt.legend()
plt.show()
print(f'AUC-ROC: {auc_roc:.4f}')

# AUC PR
precision, recall, pr = auc_pr(y_test, previsoes)
# Plotando a curva Precisão-Recall
plt.figure(figsize=(6, 6))
plt.plot(recall, precision, label=f'AUC-PR = {pr:.2f}', color='blue')
plt.xlabel('Recall')
plt.ylabel('Precisão')
plt.title('Curva de Precisão-Recall')
plt.legend()
plt.show()
print(f'AUC-PR: {pr:.4f}')

# IoU
iou = iou_score(np.array(y_test), np.array(previsoes))
print(f'IoU: {iou:.4f}')

In [ ]:
print("Carregando o modelo bound...")
modelo_bound = load_model(final_model_value_path)
print("Testando o modelo bound...")
y_test, previsoes = testar_modelo(modelo_bound, files_teste, True)

# MAE
mae = mae(y_test, previsoes)
print(f'MAE: {mae:.4f}')

# MSE
mse = mse(y_test, previsoes)
print(f'MSE: {mse:.4f}')

# RMSE
rmse = rmse(y_test, previsoes)
print(f'RMSE: {rmse:.4f}')

# R2
r2 = r2(y_test, previsoes)
print(f'R²: {r2:.4f}')

plot_predictions(y_test, previsoes)